In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# US Counties Dashboard\n",
    "Interactive regional analytics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import folium\n",
    "import plotly.graph_objects as go\n",
    "import ipywidgets as widgets\n",
    "from IPython.display import display, HTML\n",
    "from shapely.geometry import Polygon\n",
    "import geopandas as gpd\n",
    "\n",
    "print('📊 Setting up dashboard...')\n",
    "\n",
    "# Create sample counties\n",
    "counties_data = [\n",
    "    ('Los Angeles County', 'CA', (-118.9, 33.7, -117.6, 34.8)),\n",
    "    ('Cook County', 'IL', (-88.3, 41.4, -87.5, 42.1)),\n",
    "    ('Harris County', 'TX', (-95.8, 29.5, -94.9, 30.1)),\n",
    "    ('Maricopa County', 'AZ', (-113.3, 33.0, -111.0, 34.0)),\n",
    "    ('San Diego County', 'CA', (-117.6, 32.5, -116.1, 33.5))\n",
    "]\n",
    "\n",
    "# Create boundaries\n",
    "geometries = []\n",
    "names = []\n",
    "states = []\n",
    "\n",
    "for name, state, (min_lon, min_lat, max_lon, max_lat) in counties_data:\n",
    "    coords = [(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat), (min_lon, min_lat)]\n",
    "    geometries.append(Polygon(coords))\n",
    "    names.append(name)\n",
    "    states.append(state)\n",
    "\n",
    "gdf = gpd.GeoDataFrame({'NAME': names, 'STATE': states, 'geometry': geometries})\n",
    "\n",
    "# Generate sample data\n",
    "dates = pd.date_range(end='2024-01-01', periods=12, freq='M')\n",
    "data_rows = []\n",
    "\n",
    "for county, state in zip(names, states):\n",
    "    base_pop = np.random.randint(1_000_000, 10_000_000)\n",
    "    for date in dates:\n",
    "        data_rows.append({\n",
    "            'county': county,\n",
    "            'state': state,\n",
    "            'date': date,\n",
    "            'population': int(base_pop * np.random.uniform(0.98, 1.02)),\n",
    "            'gdp_millions': np.random.uniform(100, 1000),\n",
    "            'unemployment_rate': np.random.uniform(3, 8),\n",
    "            'median_income': np.random.uniform(50000, 80000)\n",
    "        })\n",
    "\n",
    "df = pd.DataFrame(data_rows)\n",
    "print(f'✅ Created data for {len(names)} counties')\n",
    "\n",
    "# Create simple controls\n",
    "county_dropdown = widgets.Dropdown(\n",
    "    options=names,\n",
    "    value=names[0],\n",
    "    description='County:'\n",
    ")\n",
    "\n",
    "metric_dropdown = widgets.Dropdown(\n",
    "    options=[('Population', 'population'), ('GDP', 'gdp_millions'), ('Unemployment', 'unemployment_rate')],\n",
    "    value='population',\n",
    "    description='Metric:'\n",
    ")\n",
    "\n",
    "def update_display(county, metric):\n",
    "    county_data = df[df['county'] == county]\n",
    "    latest = county_data[county_data['date'] == county_data['date'].max()].iloc[0]\n",
    "    \n",
    "    print(f'📊 {county}')\n",
    "    print(f'Population: {latest[\"population\"]:,}')\n",
    "    print(f'GDP: ${latest[\"gdp_millions\"]:.1f}M')\n",
    "    print(f'Unemployment: {latest[\"unemployment_rate\"]:.1f}%')\n",
    "    \n",
    "    # Create simple map\n",
    "    county_gdf = gdf[gdf['NAME'] == county]\n",
    "    center_lat = county_gdf.geometry.centroid.y.iloc[0]\n",
    "    center_lon = county_gdf.geometry.centroid.x.iloc[0]\n",
    "    \n",
    "    m = folium.Map(location=[center_lat, center_lon], zoom_start=8)\n",
    "    \n",
    "    folium.GeoJson(\n",
    "        county_gdf.iloc[0].geometry.__geo_interface__,\n",
    "        style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 2, 'fillOpacity': 0.5}\n",
    "    ).add_to(m)\n",
    "    \n",
    "    return m\n",
    "\n",
    "# Interactive widget\n",
    "interact_widget = widgets.interact(update_display, county=county_dropdown, metric=metric_dropdown)\n",
    "\n",
    "print('🗺️ Dashboard ready! Use the controls above to explore.')"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {"name": "ipython", "version": 3},
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  }
 }
}